# Geospatial Analysis with GeoPandas

In [31]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
try:
    # Intento de cargar el archivo con codificación 'latin1' (común en archivos en español)
    df = pd.read_csv(r'D:/DIPLOMATURA_CIENCIA DE DATOS_PUCP/Python Intermedio/IPRESS.csv', dtype={'UBIGEO': str}, encoding='latin1')
    print("El archivo se cargó correctamente con codificación 'latin1'.")
    
except UnicodeDecodeError:
    # Si 'latin1' falla, el siguiente intento común es con el delimitador incorrecto (e.g., ';' en lugar de ',')
    # Aunque la cabecera parece tener comas, probaremos con el separador más común en estos casos.
    try:
        df = pd.read_csv("IPRESS.csv", dtype={'UBIGEO': str}, sep=';', encoding='latin1')
        print("El archivo se cargó correctamente usando ';' como separador y codificación 'latin1'.")
    except Exception as e:
        # Si ambos fallan, mostramos el error original.
        print(f"Error: No se pudo cargar el archivo. Mensaje de error detallado: {e}")
        # En un escenario real, aquí podrías pedir al usuario el separador o la codificación, pero en este contexto simulamos el intento de arreglo más probable.
        # Para continuar con el plan, usaremos el DataFrame cargado exitosamente en el primer 'try' o en el segundo, si se produjo uno.
        raise # Detener la ejecución si no se pudo cargar el archivo

El archivo se cargó correctamente con codificación 'latin1'.


Renombrar columnas para facilitar el manejo (NORTE -> LATITUD, ESTE -> LONGITUD)

In [33]:

df.rename(columns={'NORTE': 'LATITUD', 'ESTE': 'LONGITUD'}, inplace=True)

 1. Filtrar registros que están "EN FUNCIONAMIENTO" (el "functioning status" es la columna 'Condición' según la inspección del archivo)

In [34]:

df_operacional = df[df['Condición'] == 'EN FUNCIONAMIENTO'].copy()

2. Filtrar registros con coordenadas válidas (LATITUD y LONGITUD no nulas)

Convertir las columnas de coordenadas a tipo numérico, forzando errores a NaN

In [35]:

df_operacional['LATITUD'] = pd.to_numeric(df_operacional['LATITUD'], errors='coerce')
df_operacional['LONGITUD'] = pd.to_numeric(df_operacional['LONGITUD'], errors='coerce')

Eliminar filas donde LATITUD o LONGITUD son NaN después de la conversión

In [36]:

df_filtrado_coords = df_operacional.dropna(subset=['LATITUD', 'LONGITUD'])

3. Seleccionar las columnas clave solicitadas

In [37]:

columnas_clave = [
    'Institución',
    'Nombre del establecimiento',
    'Clasificación',
    'Departamento',
    'Provincia',
    'Distrito',
    'UBIGEO',
    'Estado',
    'LATITUD',
    'LONGITUD'
]

df_final = df_filtrado_coords[columnas_clave]

# Mostrar las primeras filas del DataFrame resultante
print("DataFrame resultante (primeras 5 filas):")
print(df_final.head())

# Mostrar información sobre el DataFrame resultante
print("\nInformación del DataFrame resultante:")
print(df_final.info())

print(f"\nNúmero total de registros originales: {len(df)}")
print(f"Número de registros después de filtrar por 'EN FUNCIONAMIENTO' y coordenadas válidas: {len(df_final)}")

DataFrame resultante (primeras 5 filas):
          Institución         Nombre del establecimiento  \
1   GOBIERNO REGIONAL                             AMBATO   
2   GOBIERNO REGIONAL          SANTA ISABEL DE YUMBATURO   
6               MINSA  PUESTO DE SALUD HEROES DEL CENEPA   
12  GOBIERNO REGIONAL                      NUEVA BETANIA   
15  GOBIERNO REGIONAL                         PONGO ISLA   

                         Clasificación Departamento         Provincia  \
1   PUESTOS DE SALUD O POSTAS DE SALUD    CAJAMARCA           CUTERVO   
2   PUESTOS DE SALUD O POSTAS DE SALUD       LORETO            LORETO   
6   PUESTOS DE SALUD O POSTAS DE SALUD         LIMA              LIMA   
12  PUESTOS DE SALUD O POSTAS DE SALUD      UCAYALI  CORONEL PORTILLO   
15  PUESTOS DE SALUD O POSTAS DE SALUD   SAN MARTIN        SAN MARTIN   

             Distrito  UBIGEO    Estado    LATITUD   LONGITUD  
1          SANTA CRUZ  060611  ACTIVADO -78.858380  -6.133523  
2            PARINARI  160302  

In [39]:
print("DataFrame resultante (primeras 5 filas):")
print(df_final.head())

DataFrame resultante (primeras 5 filas):
          Institución         Nombre del establecimiento  \
1   GOBIERNO REGIONAL                             AMBATO   
2   GOBIERNO REGIONAL          SANTA ISABEL DE YUMBATURO   
6               MINSA  PUESTO DE SALUD HEROES DEL CENEPA   
12  GOBIERNO REGIONAL                      NUEVA BETANIA   
15  GOBIERNO REGIONAL                         PONGO ISLA   

                         Clasificación Departamento         Provincia  \
1   PUESTOS DE SALUD O POSTAS DE SALUD    CAJAMARCA           CUTERVO   
2   PUESTOS DE SALUD O POSTAS DE SALUD       LORETO            LORETO   
6   PUESTOS DE SALUD O POSTAS DE SALUD         LIMA              LIMA   
12  PUESTOS DE SALUD O POSTAS DE SALUD      UCAYALI  CORONEL PORTILLO   
15  PUESTOS DE SALUD O POSTAS DE SALUD   SAN MARTIN        SAN MARTIN   

             Distrito  UBIGEO    Estado    LATITUD   LONGITUD  
1          SANTA CRUZ  060611  ACTIVADO -78.858380  -6.133523  
2            PARINARI  160302  

In [40]:
df_final

,Institución,Nombre del establecimiento,Clasificación,Departamento,Provincia,Distrito,UBIGEO,Estado,LATITUD,LONGITUD
1,GOBIERNO REGIONAL,AMBATO,PUESTOS DE SALUD O POSTAS DE SALUD,CAJAMARCA,CUTERVO,SANTA CRUZ,060611,ACTIVADO,-78.858380,-6.133523
2,GOBIERNO REGIONAL,SANTA ISABEL DE YUMBATURO,PUESTOS DE SALUD O POSTAS DE SALUD,LORETO,LORETO,PARINARI,160302,ACTIVADO,-74.258139,-4.581509
6,MINSA,PUESTO DE SALUD HEROES DEL CENEPA,PUESTOS DE SALUD O POSTAS DE SALUD,LIMA,LIMA,VILLA EL SALVADOR,150142,ACTIVADO,-76.930608,-12.248699
12,GOBIERNO REGIONAL,NUEVA BETANIA,PUESTOS DE SALUD O POSTAS DE SALUD,UCAYALI,CORONEL PORTILLO,CALLERIA,250101,ACTIVADO,-74.296531,-8.398366
15,GOBIERNO REGIONAL,PONGO ISLA,PUESTOS DE SALUD O POSTAS DE SALUD,SAN MARTIN,SAN MARTIN,HUIMBAYOC,220907,ACTIVADO,-75.885812,-6.438298
...,...,...,...,...,...,...,...,...,...,...
20814,GOBIERNO REGIONAL,P.S NUEVO HUALAPAMPA,PUESTOS DE SALUD O POSTAS DE SALUD,PIURA,HUANCABAMBA,HUARMACA,200304,ACTIVADO,-79.437187,-5.789407
20815,GOBIERNO REGIONAL,P.S SAN MARTIN DE CONGOÑA,PUESTOS DE SALUD O POSTAS DE SALUD,PIURA,HUANCABAMBA,HUARMACA,200304,ACTIVADO,-79.627953,-5.644592
20816,GOBIERNO REGIONAL,P.S TUNAS,PUESTOS DE SALUD O POSTAS DE SALUD,PIURA,HUANCABAMBA,HUARMACA,200304,ACTIVADO,-79.546778,-5.498508
20817,GOBIERNO REGIONAL,C.S LIMON DE PORCUYA,CENTROS DE SALUD O CENTROS MEDICOS,PIURA,HUANCABAMBA,HUARMACA,200304,ACTIVADO,-79.537903,-5.889830


Abrir archivo geojson

In [38]:
# Define la ruta a tu archivo GeoJSON
ruta_archivo_geojson = 'D:/DIPLOMATURA_CIENCIA DE DATOS_PUCP/Python Intermedio/peru_distrital_simple.geojson'

# Importa el archivo GeoJSON directamente a un GeoDataFrame
try:
    gdf = gpd.read_file(ruta_archivo_geojson)
    
    # Muestra las primeras filas y el tipo de geometría
    print("GeoDataFrame importado con éxito:")
    print(gdf.head())
    print("\nTipo de geometría:")
    print(gdf.geom_type.head())

except Exception as e:
    print(f"Error al leer el archivo GeoJSON con GeoPandas: {e}")


GeoDataFrame importado con éxito:
   OBJECTID  IDDIST IDDPTO IDPROV                              NOMBDIST  \
0         1  230110     23   2301  CORONEL GREGORIO ALBARRACIN LANCHIPA   
1         2  230108     23   2301                              POCOLLAY   
2         3  230103     23   2301                                CALANA   
3         4  230101     23   2301                                 TACNA   
4         5  230109     23   2301                                  SAMA   

  NOMBPROV NOMBDEP DCTO    LEY         FECHA         NOM_CAP  SHAPE_LENG  \
0    TACNA   TACNA  LEY  27415    02/02/2001  ALFONSO UGARTE    0.570510   
1    TACNA   TACNA  LEY  13069    15/01/1959        POCOLLAY    0.883871   
2    TACNA   TACNA  LEY    S/N    20/08/1872          CALANA    0.446736   
3    TACNA   TACNA    -      -  EPOCA INDEP.           TACNA    2.758951   
4    TACNA   TACNA    -      -  EPOCA INDEP.       LAS YARAS    1.515506   

   SHAPE_AREA  SHAPE_LE_1  SHAPE_AR_1  AREA_MINAM  \
0    